In [1]:
import pandas as pd

### Считываем данные

In [ ]:
df = pd.read_excel('Datas/TestDataBase.xlsx')
df.dropna()
df.info()

### Создание JSON-файла для дальнейшей работы

In [ ]:
data = {}

data['intents'] = []
for i,j in zip(df['Заголовок статьи'], df['Портал самообслуживания']):
    data['intents'].append({
        'tag':i,
        'patterns': [i],
        'responses': [j],
        'context':[""]
    })
    
with open('JSONforLemm.txt', 'w', encoding='UTF-8') as f:
    json.dump(data, f,ensure_ascii=False)

### Создание игнор-символов

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!',"''",'&','«','``','»',': &', 'quot', ')','(']

with open('JSONforLemm.txt', encoding='UTF-8') as json_file:
    intents = json.load(json_file)